# general imports

In [1]:
from __future__ import division, print_function

import numpy as np
np.random.seed(42)

from scipy.constants import m_p, c, e

import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display

In [2]:
# sets the PyHEADTAIL directory etc.
try:
    from settings import *
except:
    pass

# PyHEADTAIL imports

In [3]:
from PyHEADTAIL.particles.generators import ParticleGenerator, RF_bucket_distribution, gaussian2D,gaussian2D_asymmetrical
from PyHEADTAIL.trackers.longitudinal_tracking import RFSystems
from PyHEADTAIL.trackers.rf_bucket import RFBucket
from PyHEADTAIL.trackers.transverse_tracking import TransverseMap
from PyHEADTAIL.monitors.monitors import BunchMonitor, SliceMonitor, ParticleMonitor
from PyHEADTAIL.trackers.detuners import Chromaticity, AmplitudeDetuning
import sys
from PyHEADTAIL.particles import generators,particles
from PyHEADTAIL.particles.generators import RFBucketMatcher
from PyHEADTAIL.cobra_functions.stats import emittance

PyHEADTAIL v1.14.1




# Setting up the machine and beam parameters

In [4]:
# General

nturns = 100 #200  #20000 #2**16
npart =5000
A=1
Q=-1

Ekin_per_nucleon = 100000 #0.2e9 # in eV
intensity =4.5e6 #0.625e11
nmass=0.9383
mass_eV = A * nmass * 1e9
mass_MeV = A * nmass * 1e3
mass = mass_eV * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e # in SI units
gamma = np.sqrt(1 + (p0 / (mass * c))**2)
beta = np.sqrt(1 - gamma**-2)

# Machine
C = 30.4056
R = C/(2*np.pi)

alpha  = 0.25835
eta    = alpha - 1/gamma**2
V      = [100]
h      = [1]
phi   = [np.pi]
frequency1=0.143938 #MhZ

# Acceleration
T0            = C/(beta*c)
normalisation = 1/C * e/p0 * T0

epsx_rms_fin = 2.5e-6 #35e-6 / 4 # geometrical emittances
epsy_rms_fin = 2.5e-6 #15e-6 / 4

limit_n_rms_x = 2  #3.5#2
limit_n_rms_y = 2#3.5#2
limit_n_rms_z = 2 #3.5#3.4

sig_z = 0.3284 #58 / 4. # in m
sig_dp =1e-4 #0.5e-3e
epsx_gauss = epsx_rms_fin #* 1.778 RMS adjustment not needed if limit_n_rms are >>2!
epsy_gauss = epsy_rms_fin #* 1.82 RMS adjustment not needed if limit_n_rms are >>2!

epsn_x = epsx_gauss * beta * gamma
epsn_y = epsy_gauss * beta * gamma

beta_z = sig_z / sig_dp

epsn_z = sig_z * sig_dp * 4 * np.pi * p0/e

p_increment = sig_dp * p0



# Dictionary for IBS calculations

In [5]:
beam_dict={'Z':Q,
           'mass':mass_MeV,
           'KE':Ekin/1e+6,
           'emit_x':epsx_gauss,
           'emit_y':epsx_gauss,
           'dp_p':sig_dp,
           'N_ptcl':intensity,
           'sigma_s0':sig_z
            }

# Preparing Twiss files

In [6]:
from cpymad.madx import Madx
qx = 2.454 #18.95#84 #tune_range_qx[0]
qy = 1.416  #18.94#73 #tune_range_qy[0]
install_apertures = False
apply_errors = False
Thin=False
pystl_device = "opencl:0.0"
e_seed = 24

qqx, qqy = int(np.round((qx%1) * 100)), int(np.round((qy%1) * 100))
filename_errors = "errors_{qqx}_{qqy}_{eseed:d}".format(
            qqx=qqx, qqy=qqy, eseed=e_seed)

madx=Madx()
madx.options.echo = False
madx.options.warn = False
madx = Madx(stdout=False)
madx.call('elena_simplified_origin.madx')


madx.command.beam(particle='ion', mass=A*nmass, charge=Q, energy=Etot) # /Q for RF to have proton beam!

title="Thick elements"

madx.use(sequence='ELENA') 
        
if Thin==False:
    
    madx.input('''match, sequence=ELENA; 
    global, sequence=ELENA, q1={qx}, q2={qy};
    vary, name=kq1, step=0.00001;
    vary, name=kq2, step=0.00001;
    lmdif, calls=500, tolerance=1.0e-10;
    endmatch;
    '''.format(qx=qx, qy=qy)
    )
    
    twiss = madx.twiss();

madx_seq = madx.sequence.ELENA
print(f"MAD-X summary: \n Qx= {twiss.summary.q1} \n Qy= {twiss.summary.q2} \n dQx={twiss.summary.dq1*beta}")
qx_act=twiss.summary.q1
qy_act=twiss.summary.q2
dq1=twiss.summary.dq1*beta
dq2=twiss.summary.dq2*beta


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.06.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2020.09.01               +
  + Execution date: 2021.06.01 20:19:22      +
  ++++++++++++++++++++++++++++++++++++++++++++
MAD-X summary: 
 Qx= 2.454 
 Qy= 1.416 
 dQx=-3.1574408714289386


# IBS functions

In [7]:
from PyHEAD_IBS import IBS_Martini_rates,IBS_BM_rates,IBS_kick

In [8]:
IBS_Martini=IBS_Martini_rates(twiss,beam_dict,True,60,60,100,12.5,1)
print(f'Rates X {IBS_Martini.rsx} # Rates Y {IBS_Martini.rsy} # Rates Z {IBS_Martini.rss} #')

Rates X 0.22132545210160895 # Rates Y -0.31346560046982186 # Rates Z 51.2198147405692 #


In [9]:
IBS_BM=IBS_BM_rates(twiss,beam_dict,True,60,60,100,12.5,1)
print(f'Rates X {IBS_BM.rsx} # Rates Y {IBS_BM.rsy} # Rates Z {IBS_BM.rss} #')

Rates X 0.22069657967379055 # Rates Y -0.3062356075066439 # Rates Z 51.69187245553312 #


In [ ]:
chroma = Chromaticity(Qp_x=[dq1], Qp_y=[dq2]) # note the lists!

## Mean optics from MAD-X IBS calculations

In [ ]:
betx   =   4.20436E+00    bety   =   3.48406E+00    Dx  =  1.21510E+00    Dy  =  0.00000E+00
alfx   =   4.01651E-16    alfy   =   4.81982E-16    Dpx = -4.38165E-17    Dpy =  0.00000E+00
1/betx =   5.05807E-01    1/bety =   2.93247E-01


In [ ]:
n_segments = 1
C = twiss.summary.length
R = C / (2.*np.pi)
s = np.arange(0, n_segments + 1) * C / n_segments

alpha_x_inj = 0.
alpha_y_inj = 0.

beta_x_inj =4.20436 #R/qx_act
beta_y_inj =3.48406 #R/qy_act

D_x_inj=1.2151
D_y_inj=0

alpha_x = alpha_x_inj * np.ones(n_segments)
beta_x = beta_x_inj * np.ones(n_segments)
alpha_y = alpha_y_inj * np.ones(n_segments)
beta_y = beta_y_inj * np.ones(n_segments)

D_x = np.ones(n_segments)*D_x_inj
D_y = np.ones(n_segments)*D_y_inj

dq1=twiss.summary.dq1*beta
dq2=twiss.summary.dq2*beta

trans_map_smooth = TransverseMap(
s, alpha_x, beta_x, D_x, alpha_y, beta_y, D_y, qx_act, qx_act, [chroma])

In [ ]:
longitudinal_map = RFSystems(
    C, h, V, phi, [alpha], gamma, 
    p_increment=0, charge=-e, mass=m_p,D_x=D_x_inj, D_y=0)

rfbucket = longitudinal_map.get_bucket(gamma=gamma)

In [ ]:
np.random.seed(3)

pyht_beam = generators.generate_Gaussian6DTwiss(
    npart, 1, charge, mass, C, gamma,
    alpha_x_inj, alpha_y_inj,beta_x_inj, beta_y_inj,
    beta_z, epsn_x, epsn_y, epsn_z,
    dispersion_x=D_x_inj, #D_x_0 if D_x_0 else None,
    dispersion_y=0, #D_y_0 if D_y_0 else None,
    limit_n_rms_x=limit_n_rms_x**2, limit_n_rms_y=limit_n_rms_y**2, 
    limit_n_rms_z=limit_n_rms_z**2,
)


In [ ]:
epsn_x_ratio_sq = np.sqrt(epsn_x / pyht_beam.epsn_x())
epsn_y_ratio_sq = np.sqrt(epsn_y / pyht_beam.epsn_y())
dp_ratio=sig_dp / np.std( pyht_beam.dp)
sigz_ratio=sig_z/np.std( pyht_beam.z)

pyht_beam.x *= epsn_x_ratio_sq
pyht_beam.xp *= epsn_x_ratio_sq
pyht_beam.y *= epsn_y_ratio_sq
pyht_beam.yp *= epsn_y_ratio_sq
pyht_beam.z *= sigz_ratio
pyht_beam.dp *= dp_ratio

In [ ]:
import h5py as hp
def read_all_data(bfile,data):
    bunchdata = hp.File(bfile + '.h5')

    # Bunchdata
    bdata = bunchdata['Bunch']
    
    n_turns = len(bdata[data])
    _ = np.empty(n_turns)
    for key in bdata.keys():
        _[:] = bdata[key][:]
    
    bunchdata.close()

def read_n_plot_data(bfile,nturns,data,factor):
    bunchdata = hp.File(bfile + '.h5')

    fig = plt.figure(figsize=(16, 16))
    ax1 = fig.add_subplot(311)

    ax1.plot(bunchdata['Bunch'][data][:nturns]*factor)
    ax1.set_xlabel('turns', fontsize=20)
    ax1.set_ylabel(f'{data} of bunch', fontsize=20)
    ax1.set_title('BunchMonitor', fontsize=20)
    
    plt.tight_layout()
    
    bunchdata.close()

In [ ]:
# Monitors
bunch_filename = 'bunch_mon'
particle_filename = 'particle_mon'

bunch_monitor = BunchMonitor(
    filename=bunch_filename, n_steps=nturns, 
    parameters_dict={'Q_x': qx_act}, write_buffer_every=2
)
particle_monitor = ParticleMonitor(
    filename=particle_filename, stride=10, 
    parameters_dict={'Q_x': qx_act}
)

In [ ]:
import os
os.remove(bunch_filename + '.h5')
os.remove(particle_filename + '.h5part')

In [ ]:
map_ = list(trans_map_smooth)+[longitudinal_map]
dt=twiss.summary.length/(pyht_beam.beta*c)
rxl=[]
ryl=[]
rsl=[]
time=[]

for i in range(nturns):
    t=i+1
    for m_ in map_:
        m_.track(pyht_beam)
   #Update beam dictionary     
    beam_dict={'Z':Q,
    'mass':mass_MeV,
    'KE':Ekin/1e+6,
    'emit_x':pyht_beam.epsn_x()/pyht_beam.betagamma,
    'emit_y':pyht_beam.epsn_y()/pyht_beam.betagamma,
    'dp_p':pyht_beam.sigma_dp(),
    'N_ptcl':intensity,
    'sigma_s0':pyht_beam.sigma_z()            
    }  
   #Calculate rates
    IBS=IBS_Martini_rates(twiss,beam_dict,True,60,60,100,12.5,1)
    print(f'Time =  {dt*t}  Rates X {IBS.rsx} # Rates Y {IBS.rsy} # Rates Z {IBS.rss} #')
    
    growth_rx=IBS.rsx
    growth_ry=IBS.rsy
    growth_rs=IBS.rss
    
    rxl.append(growth_rx)
    ryl.append(growth_ry)
    rsl.append(growth_rs)
    time.append(dt*t)
    
    #Apply IBS kick
    IBSMap=IBS_kick(growth_rx,growth_ry,growth_rs, dt ,Dx=D_x_inj,Dy=0)
    IBSMap.track(pyht_beam)
    
    bunch_monitor.dump(pyht_beam)  
    

In [ ]:
with hp.File(bunch_filename + '.h5') as bunchdata:
    print(bunchdata['Bunch']['epsn_x'][0]/pyht_beam.betagamma)

In [ ]:
with hp.File(bunch_filename + '.h5') as bunchdata:

    fig = plt.figure(figsize=(8,5))

    data='ryl'
    plt.plot(rsl)
    plt.xlabel('turns', fontsize=20)
    plt.ylabel(f'{data} of bunch', fontsize=20)
    plt.title('BunchMonitor', fontsize=20)
    plt.twinx()
    plt.plot((bunchdata['Bunch']['sigma_z'][0:nturns]))

In [ ]:
with hp.File(bunch_filename + '.h5') as bunchdata:
    
    exb=bunchdata['Bunch']['epsn_x'][0:nturns]
    eyb=bunchdata['Bunch']['epsn_y'][0:nturns]
    sigzb=bunchdata['Bunch']['sigma_z'][0:nturns]
    b=list(zip( time,exb,eyb,dpl,sigzb,rxl,ryl,rsl))    
    np.savetxt("PyH_everything after.csv", np.array(b), delimiter="\t")   

In [ ]:
fig = plt.figure(figsize=(16, 16))
ax1 = fig.add_subplot(311)

data='ryl'
ax1.plot(rxl)

#ax1.plot(bunchdata['Bunch'][data][:nturns]*1)
ax1.set_xlabel('turns', fontsize=20)
ax1.set_ylabel(f'{data} of bunch', fontsize=20)
ax1.set_title('BunchMonitor', fontsize=20)

In [ ]:
fig = plt.figure(figsize=(16, 16))
ax1 = fig.add_subplot(311)

data='ryl'
ax1.plot(ryl)
#ax1.plot(bunchdata['Bunch'][data][:nturns]*1)
ax1.set_xlabel('turns', fontsize=20)
ax1.set_ylabel(f'{data} of bunch', fontsize=20)
ax1.set_title('BunchMonitor', fontsize=20)